In [1]:
!apt-get update -qq
!apt-get install -y -qq poppler-utils

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
!pip install -q "mineru-vl-utils[transformers]" transformers pillow pdf2image torch --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00


In [3]:
import os
import json
import io
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path
import torch

# MinerU client
from mineru_vl_utils import MinerUClient

# Hugging Face model id
MODEL_ID = "opendatalab/MinerU2.5-2509-1.2B"

# Check device
USE_CUDA = torch.cuda.is_available()
print("CUDA available:", USE_CUDA)

CUDA available: True


In [4]:
# Colab cell 3: load model & processor (transformers backend)
# The model card suggests using the mineru-vl-utils wrapper; this matches their example. :contentReference[oaicite:2]{index=2}
from transformers import AutoProcessor
# For newer transformers versions the model class is Qwen2VLForConditionalGeneration
# But mineru-vl-utils can accept a model object or a vllm client; we will load a transformers model (CPU/GPU aware).

# Try to load model; if memory is constrained, consider loading with low_cpu_mem_usage or quantized checkpoints.
try:
    from transformers import Qwen2VLForConditionalGeneration
    model = Qwen2VLForConditionalGeneration.from_pretrained(MODEL_ID, device_map="auto")
except Exception as e:
    print("Auto device_map load failed or not supported, falling back to CPU. Error:", e)
    from transformers import Qwen2VLForConditionalGeneration
    model = Qwen2VLForConditionalGeneration.from_pretrained(MODEL_ID, device_map={"": "cpu"})

processor = AutoProcessor.from_pretrained(MODEL_ID, use_fast=True)

# create the MinerU client using transformers backend
client = MinerUClient(backend="transformers", model=model, processor=processor)
print("Model and client ready.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

Model and client ready.


In [5]:
# Colab cell 4: helper - convert pdf pages to images
def pdf_to_images(pdf_path, dpi=200, output_folder="pdf_images"):
    """
    Convert each PDF page to a PIL.Image and save to output_folder.
    Returns list of PIL.Image objects (in memory) and list of file paths.
    """
    os.makedirs(output_folder, exist_ok=True)
    pil_pages = convert_from_path(pdf_path, dpi=dpi)
    paths = []
    for i, page in enumerate(pil_pages, start=1):
        p = Path(output_folder) / f"page_{i:03d}.png"
        page.save(p, "PNG")
        paths.append(str(p))
    return pil_pages, paths


In [6]:
# Colab cell 5: main function - process PDF and create JSON
def pdf_to_mineru_json(pdf_path, output_json_path="output.json", dpi=200, verbose=True):
    images, image_paths = pdf_to_images(pdf_path, dpi=dpi)
    all_pages = []
    for i, img in enumerate(images, start=1):
        if verbose:
            print(f"Processing page {i}/{len(images)} ...")
        # mineru client expects a PIL.Image
        extracted_blocks = client.two_step_extract(img)  # synchronous API per model card. :contentReference[oaicite:3]{index=3}

        # extracted_blocks is typically a list/dict structure describing layout blocks, text, bbox, type, etc.
        # We'll keep it as-is and add metadata about page index and source image name.
        page_result = {
            "page_index": i,
            "image_path": image_paths[i-1],
            "extracted_blocks": extracted_blocks
        }
        all_pages.append(page_result)

    # Save JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump({"pdf": os.path.basename(pdf_path), "pages": all_pages}, f, ensure_ascii=False, indent=2)

    print("Saved JSON to", output_json_path)
    return output_json_path


In [7]:
# Colab cell 6: upload and run (interactive part)
# Use Colab's file upload widget or mount Drive. For small testing, use upload.
from google.colab import files
uploaded = files.upload()  # choose your PDF file from local machine

# take the first uploaded file
pdf_file = list(uploaded.keys())[0]
print("Uploaded:", pdf_file)

# run conversion
out_json = pdf_to_mineru_json(pdf_file, output_json_path="mineru_output.json", dpi=200)
# provide a link to download in Colab
files.download(out_json)


Saving TBS_Handbook-2022.pdf to TBS_Handbook-2022.pdf
Uploaded: TBS_Handbook-2022.pdf
Processing page 1/30 ...


Predict: 100%|██████████| 10/10 [00:04<00:00,  2.34it/s]


Processing page 2/30 ...


Predict: 100%|██████████| 4/4 [00:05<00:00,  1.27s/it]


Processing page 3/30 ...


...

Processing page 30/30 ...


Predict: 100%|██████████| 3/3 [00:01<00:00,  1.64it/s]

Saved JSON to mineru_output.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>